<a href="https://colab.research.google.com/github/HugoFromont/Project/blob/master/Image_classification/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification d'images

In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

from google.colab import drive
drive.mount('/content/gdrive')

## Importation des données

In [ ]:
!unzip '/content/gdrive/My Drive/data_google_colab/Dogs_vs_Cats/train.zip'

files = os.listdir("/content/train/")
files[:4]

In [ ]:
# Echantillonnage
from sklearn.model_selection import train_test_split

train, valid = train_test_split(files, test_size=0.2, random_state=123)

train = pd.DataFrame(train, columns = ['id'])
valid = pd.DataFrame(valid, columns = ['id'])

# Création du label
def create_label(id):
  return('1' if 'cat' in id else '0')

train['label'] = train.id.apply(create_label)
valid['label'] = valid.id.apply(create_label)

train.head()

## Création du modèle

### Construction d'un CNN

In [ ]:
input_img = tf.keras.Input(shape=(112,112,3))
x = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)

x = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)

x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)

x = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(500)(x)
x = tf.keras.layers.Dense(1, activation = "sigmoid")(x)

model = tf.keras.Model(input_img, x)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

### Entraînement du modèle

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train, 
    "train/", 
    x_col='id',
    y_col='label',
    target_size=((112,112)),
    class_mode='binary',
    batch_size=100
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    valid, 
    "train/", 
    x_col='id',
    y_col='label',
    target_size=((112,112)),
    class_mode='binary',
    batch_size=100
)

In [ ]:
history_M1 = model.fit_generator(
    train_generator, 
    epochs=100,
    validation_data=validation_generator,
    shuffle = True
)

## Ajout de régularisation

### Drop Out

In [ ]:
input_img = tf.keras.Input(shape=(112,112,3))
x = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(500)(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation = "sigmoid")(x)

model = tf.keras.Model(input_img, x)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

In [ ]:
history_M3 = model.fit_generator(
    train_generator, 
    epochs=100,
    validation_data=validation_generator,
    shuffle = True
)

### Pré-procesing

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train, 
    "train/", 
    x_col='id',
    y_col='label',
    target_size=((112,112)),
    class_mode='binary',
    batch_size=100
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    valid, 
    "train/", 
    x_col='id',
    y_col='label',
    target_size=((112,112)),
    class_mode='binary',
    batch_size=100
)

In [ ]:
history_M2 = model.fit_generator(
    train_generator, 
    epochs=100,
    validation_data=validation_generator,
    shuffle = True,
    callbacks=[cp_callback]
)

## Transfert Learning

In [ ]:
pre_training_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(112,112,3))

model = tf.keras.models.Sequential()
model.add(pre_training_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['acc'])

model.summary()

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
model.fit_generator(train_datagen.flow(features,targets, batch_size=65),
                    steps_per_epoch=len(targets)/65, epochs=3,
                    shuffle = True, 
                    validation_data = valid_datagen.flow(valid_features, valid_targets, batch_size=65)
                    )